<a href="https://colab.research.google.com/github/ncoliver/beginnerAICourse/blob/main/finetuneChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 44.6 MB/s eta 0:00:00


In [ ]:
# ✅ 2️⃣ Imports
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

# ✅ 3️⃣ Load model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token   # important for padding

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# ✅ 4️⃣ Load and preprocess dataset
dataset = load_dataset("json", data_files="basketball.json")

def tokenize(example):
    # Concatenate question and answer text
    text = example["question"] + "\n" + example["answer"] + tokenizer.eos_token
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    # Add labels for supervised fine-tuning
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_ds = dataset.map(tokenize)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
# ✅ 5️⃣ Configure LoRA for parameter-efficient fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
# ✅ 6️⃣ Define training arguments
training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=100,  # start small; increase after verifying
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit"  # memory-efficient optimizer
)

In [ ]:
# ✅ 7️⃣ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"]
)

In [ ]:
# ✅ 8️⃣ Train
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ncoliver (ncoliver-mosu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,5.515700
20,5.444200
30,5.323900
40,5.306400
50,5.276700
60,5.268800
70,5.279500
80,5.266000
90,5.279900
100,5.286800


TrainOutput(global_step=500, training_loss=5.28146346282959, metrics={'train_runtime': 1260.1853, 'train_samples_per_second': 1.587, 'train_steps_per_second': 0.397, 'total_flos': 4.3708833595392e+16, 'train_loss': 5.28146346282959, 'epoch': 100.0})

In [ ]:
# ✅ Save the fine-tuned LoRA adapter
model.save_pretrained("./chatbot_model")
print("✅ LoRA adapter saved to ./chatbot_model")

✅ LoRA adapter saved to ./chatbot_model


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Load base model + tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map="auto", torch_dtype="auto"
)

# Load adapter (now exists!)
model = PeftModel.from_pretrained(base_model, "./chatbot_model")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain basketball fouls: What are major and minor infractions?
Major fouls disrupt play and give an advantage. They include shooting, driving, and defensive three seconds. Minor fouls are less severe, such as blocking, pushing, and lingering in the lane. Both affect play length and team reputation.


In [ ]:
print(pipe("Explain systematic basketball:", max_new_tokens=150)[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain systematic basketball:systematic How is systematic basketball:systematic built?
Systematic basketball:systematic is built from core principles, repeated until second nature. It’s structured like a pyramid, with fundamentals at the base supporting progressively advanced tactics.


In [ ]:
def chatbot(prompt, chat_history=[]):
    """Generate model response given user input and prior conversation."""
    history_text = "\n".join([f"User: {u}\nAssistant: {a}" for u, a in chat_history])
    full_prompt = f"{history_text}\nUser: {prompt}\nAssistant:"

    output = pipe(
        full_prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )[0]["generated_text"]

    # Extract only the assistant part
    response = output.split("Assistant:")[-1].strip()
    chat_history.append((prompt, response))
    return chat_history, chat_history

In [ ]:
def save_feedback(rating, comment):
    with open("feedback.csv", "a") as f:
        f.write(f"{rating},{comment}\n")
    return "⭐ Thanks for your feedback!"

with gr.Blocks() as demo:
    gr.Markdown("# Teaching Chatbot with Feedback")
    chat = gr.Chatbot()
    text = gr.Textbox(label="Ask me something!")
    text.submit(lambda m, h: (h + [(m, f"Response: {m}")], ""), [text, chat], [chat, text])

    gr.Markdown("## 💬 Rate your experience")
    rating = gr.Slider(1, 5, value=5, step=1, label="Rate (1=Poor, 5=Excellent)")
    comment = gr.Textbox(label="Optional comment")
    submit = gr.Button("Submit")
    output = gr.Textbox(label="Status", interactive=False)

    submit.click(save_feedback, [rating, comment], output)

demo.launch()

/tmp/ipython-input-1733990398.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a1c97574d77632c3f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install nbformat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!zip -r /content/chatbot_model.zip /content/chatbot_model
!mv /content/chatbot_model.zip /content/drive/MyDrive/chatbot_model.zip

  adding: content/chatbot_model/ (stored 0%)
  adding: content/chatbot_model/adapter_model.safetensors (deflated 7%)
  adding: content/chatbot_model/runs/ (stored 0%)
  adding: content/chatbot_model/runs/Oct19_22-11-43_f1abb24b1c58/ (stored 0%)
  adding: content/chatbot_model/runs/Oct19_22-11-43_f1abb24b1c58/events.out.tfevents.1760911906.f1abb24b1c58.4027.0 (deflated 65%)
  adding: content/chatbot_model/checkpoint-415/ (stored 0%)
  adding: content/chatbot_model/checkpoint-415/adapter_model.safetensors (deflated 7%)
  adding: content/chatbot_model/checkpoint-415/optimizer.pt (deflated 11%)
  adding: content/chatbot_model/checkpoint-415/rng_state.pth (deflated 26%)
  adding: content/chatbot_model/checkpoint-415/training_args.bin (deflated 53%)
  adding: content/chatbot_model/checkpoint-415/README.md (deflated 66%)
  adding: content/chatbot_model/checkpoint-415/scheduler.pt (deflated 61%)
  adding: content/chatbot_model/checkpoint-415/scaler.pt (deflated 64%)
  adding: content/chatbot_m